In [ ]:
# creating a trasnformer for my data to generate predictions of audio

In [1]:
#imports:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
class AudioFrequencyDataset(Dataset):
    def __init__(self, npz_files):
        self.data = []
        for file in npz_files:
            npz_data = np.load(file, allow_pickle=True)
            for key in npz_data:
                self.data.append(npz_data[key])
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        seq = self.data[idx]
        
        print(f"Index {idx} - Original sequence shape: {seq.shape}")  # Debugging line
    
        seq = np.pad(seq, ((0, max(0, 128 - len(seq))), (0, 0)), mode='constant')[:128]
    
        print(f"Index {idx} - Padded sequence shape: {seq.shape}")  # Debugging line
    
    
        return torch.tensor(seq, dtype=torch.float32)

# import torch
# import numpy as np
# from torch.utils.data import Dataset, DataLoader

# class AudioFrequencyDataset(Dataset):
#     def __init__(self, tokenized_data):
#         self.data = tokenized_data  # List of tokenized sequences

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         seq = self.data[idx]
#         seq = np.pad(seq, ((0, max(0, 128 - len(seq))), (0, 0)), mode='constant')[:128]  # Pad/truncate
#         return torch.tensor(seq, dtype=torch.float32)


In [4]:
npz_files = ["/home/renzo/projects/stempalooza/procesed/00" + str(i)+"_tokens.npz" for i in range(0,10)]  # Replace with actual paths
npz_files += ["/home/renzo/projects/stempalooza/procesed/0" + str(i)+"_tokens.npz" for i in range(10,100)]  # Replace with actual paths
npz_files += ["/home/renzo/projects/stempalooza/procesed/" + str(i)+"_tokens.npz" for i in range(100,156)]  # Replace with actual paths

dataset = AudioFrequencyDataset(npz_files)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
for batch in dataloader:
    print(batch.shape)  # This triggers __getitem__ for multiple indices


Index 7045 - Original sequence shape: (128, 5, 3)


ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (2,2)  and requested shape (3,2)

In [7]:
# class GPTForAudio(nn.Module):
#     def __init__(self, input_dim=5, model_dim=128, num_heads=4, num_layers=4, ff_dim=256):
#         super().__init__()
#         self.embedding = nn.Linear(input_dim, model_dim)
#         encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, dim_feedforward=ff_dim)
#         self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
#         self.fc_out = nn.Linear(model_dim, input_dim)
    
#     def forward(self, x):
#         x = self.embedding(x)
#         x = self.transformer(x)
#         x = self.fc_out(x)
#         return x

import torch.nn as nn
import torch.nn.functional as F

class AudioTransformer(nn.Module):
    def __init__(self, input_dim=3, embed_dim=256, num_heads=8, num_layers=6):
        super(AudioTransformer, self).__init__()
        self.embedding = nn.Linear(input_dim, embed_dim)
        self.pos_encoder = nn.Parameter(torch.randn(1, 128, embed_dim))
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(embed_dim, num_heads),
            num_layers
        )
        self.decoder = nn.Linear(embed_dim, input_dim)  # Output predicted frequencies

    def forward(self, x):
        x = self.embedding(x) + self.pos_encoder
        x = self.transformer(x)
        return self.decoder(x)


In [25]:
# # Initialize model, loss function, and optimizer
# model = GPTForAudio().to(device)
# criterion = nn.MSELoss()
# optimizer = optim.AdamW(model.parameters(), lr=0.001)

# # Training loop
# num_epochs = 10
# for epoch in range(num_epochs):
#     print('new epoch starting')
#     model.train()
#     epoch_loss = 0
#     for batch in dataloader:
#         print('new batch')
#         batch = batch.to(device)
#         optimizer.zero_grad()
#         output = model(batch)
#         loss = criterion(output, batch)
#         loss.backward()
#         optimizer.step()
#         epoch_loss += loss.item()
#     print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss / len(dataloader)}")

# # Save trained model
# torch.save(model.state_dict(), "gpt_audio_model.pth")
# print("Model training complete and saved!")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train(model, dataloader, epochs=10, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for batch in dataloader:
            batch = batch.to(device)
            optimizer.zero_grad()
            output = model(batch)
            loss = criterion(output, batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

# Example usage
tokenized_audio = [np.random.rand(np.random.randint(100, 130), 3) for _ in range(1000)]
dataset = AudioFrequencyDataset(tokenized_audio)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

model = AudioTransformer().to(device)
train(model, dataloader)


new epoch starting


/usr/lib/python3.13/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (2,2)  and requested shape (3,2)